# Eccentricity convergence test

In [ ]:
import celmech as cm
from celmech.canonical_transformations import reduce_hamiltonian
from celmech.miscellaneous import getOmegaMatrix
import rebound as rb
import numpy as np
from matplotlib import pyplot as plt
import sympy as sp
import sys
sys.path.append("../code/")
from DisturbingFunctionSplittingMethod import H0soln,H1soln

# Generate a simulation of equal mass, evenly spaced planets.
def get_sim(m,Npl,pratio,exfrac):
    alpha = pratio**(-2/3)
    ex = np.min((1/alpha-1,1-alpha)) # orbit-crossing eccentricity
    ecc = exfrac * ex
    sim = rb.Simulation()
    sim.add(m=1)
    P = 1
    for i in range(Npl):
        sim.add(m=m,P=P,l=np.random.uniform(-np.pi,np.pi),e=ecc,pomega= np.random.uniform(-np.pi,np.pi))
        P*= pratio
    sim.move_to_com()
    return sim

In [ ]:
def sim2stuff(sim,jres,kres):
    pvars = cm.Poincare.from_Simulation(sim)
    pham = cm.PoincareHamiltonian(pvars)
    Hkep = pham.H.copy()
    pham_inits = pham.state.values
    jres,kres = 4,1
    for i in range(1,pham.N-1):
        pham.add_MMR_terms(jres,kres,max_order=2,indexIn=i,indexOut=i+1,inclinations=False)

    # Reduce to eliminate dependence on inclination variables...
    Hpert = reduce_hamiltonian(pham)
    # ...and subrtract Keplerian piece to create a Hamiltonian that is just the perturbation terms
    Hpert.H += -1 * Hkep

    Npl = pham.N-1

    jac=Hpert.calculate_jacobian()

    # indicies of lambda,Lambda, and x variables
    l_indx = [2*j for j in range(Npl)]
    L_indx = [2*Npl + 2*j for j in range(Npl)]
    evar_indx = [1 + 2*j for j in range(Npl)] + [2*Npl+ 1 + 2*j for j in range(Npl)]

    # Atilde matrix
    Atilde = np.array([[jac[i,j] for j in evar_indx] for i in evar_indx])

    # variable symbols
    evarsymbols = [Hpert.qp_vars[j] for j in evar_indx]
    L_symbols = [Hpert.qp_vars[j] for j in L_indx]
    l_symbols = [Hpert.qp_vars[j] for j in l_indx]

    # set ecc variables to zero to get btilde vector
    ezero_rule={s:0 for s in evarsymbols}
    btilde = np.array([Hpert.N_flow[i].xreplace(ezero_rule).subs(Hpert.qp) for i in evar_indx],dtype = float)

    # Initial values
    x0 = np.array([Hpert.state.values[i] for i in evar_indx])
    l0 = np.array([Hpert.state.values[i] for i in l_indx])
    L0 = np.array([Hpert.state.values[i] for i in L_indx])
    
    OmegaN = getOmegaMatrix(Npl)
    
    # get A as a function of lambdas
    Amtrx = sp.Matrix(2*Npl,2*Npl,lambda i,j: sp.diff(Hpert.N_H,evarsymbols[i],evarsymbols[j]))
    Afn = sp.lambdify(l_symbols,Amtrx)

    # Get b as a function of lambdas
    bvec = sp.Matrix([sp.diff(Hpert.N_H,evar).xreplace(ezero_rule) for evar in evarsymbols])
    bfn = sp.lambdify(l_symbols,bvec)

    # Derivatives of A and b as a function of lambda
    grad_A = []
    grad_b = sp.Matrix(Npl,2*Npl,lambda i,j: sp.diff(bvec[j],l_symbols[i]))
    for l_symbol in l_symbols:
        grad_A.append(sp.diff(Amtrx,l_symbol))
    grad_Afn = sp.lambdify(l_symbols,grad_A)
    grad_bfn = sp.lambdify(l_symbols,grad_b)
    GMmvec = pvars.G * pvars.G * np.array([p.M**2 * p.mu**3 for p in pvars.particles[1:]])
    
    return GMmvec,Afn,bfn,grad_Afn,grad_bfn,OmegaN,pham,x0,l0,L0

In [ ]:
def experiment(sim,T,Nsteps):
    GMmvec,Afn,bfn,grad_Afn,grad_bfn,OmegaN,pham,x0,l0,L0 = sim2stuff(sim,4,1)
    Npl = pham.N - 1 
    to_state = lambda x,l,L: np.array([(l[i],x[i],0) for i in range(Npl)] + [(L[i],x[Npl+i],0) for i in range(Npl)]).reshape(-1)

    H0step = lambda x,l,L,h: H0soln(x,l,L,h,GMmvec)
    H1step = lambda x,l,L,h: H1soln(x,l,L,h,Afn,bfn,grad_Afn,grad_bfn,OmegaN,Npl)
    
    all_energy = []
    for k,Nstep in enumerate(Nsteps):
        x,l,L = x0.copy(),l0.copy(),L0.copy()
        h = T / Nstep
        energy = np.zeros(Nstep)
        for i in range(Nstep):
            energy[i] = pham.H_func(*to_state(x,l,L))
            x,l,L = H0step(x,l,L,0.5*h)
            x,l,L = H1step(x,l,L,h)
            x,l,L = H0step(x,l,L,0.5*h)
        all_energy.append(energy)
    return all_energy

In [ ]:
n_Nsteps = 3
n_Deltas = 3
Nsteps = np.logspace(1,3,n_Nsteps).astype(int)
Deltas = np.linspace(0,0.05,n_Deltas)
energy_err_vs_dt = np.zeros((n_Deltas,n_Nsteps))
for i,Delta in enumerate(Deltas):
    energy_results = experiment(get_sim(1e-5,5,4 * (1+Delta) / 3,0.3),400.,Nsteps)
    energy_err_vs_dt[i] = [np.sqrt(np.var(res/np.mean(res))) for res in energy_results]

In [ ]:
dts = 400./Nsteps
for i,Delta in enumerate(Deltas):
    plt.plot(dts,energy_err_vs_dt[i],'s-',label="{:0.3f}".format(Delta))
plt.plot(400./Nsteps,3e-10*(400./Nsteps)**2,'k--',label = "$\propto dt^2$")
plt.xscale('log')
plt.yscale('log')
plt.xlabel("$dt$",fontsize=16)
plt.ylabel("$(\Delta E / E)_\mathrm{rms}$",fontsize=16)
plt.legend()